In [128]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [129]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [130]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [143]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, num_layer, train=True):
        model = self.models[num_layer-1]
        Wxh, Whh, Why = model['Wxh'], model['Whh'], model['Why']
        bh, by = model['bh'], model['by']

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = X, Whh, h, hprev, y, h_cache, y_cache, Wxh

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, dy, dh_next, cache):
        X, Whh, h, hprev, y, h_cache, y_cache, Wxh = cache

        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        dh_next = dh @ Whh.T
        dX = dh @ Wxh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)

        return grad, dh_next, dX

    def _init_model(self, D, C, H):
        self.models = []
        total_num_layers = 1
        for l in range(total_num_layers):
            model = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
            )
            self.models.append(model)
            
    def train_step_fwd(self, X_train, y_train, h):
        ys, caches = [], []
        h_ = h.copy()
#         h_2 = h.copy()
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            x = X_one_hot.reshape(1, -1)
            x, h_, cache = self.forward(x, h_, train=True, num_layer=1)
#             x, h_2, cache2 = self.forward(x, h_2, train=True, num_layer=2)
            ys.append(x)
            caches.append(cache)
#             caches.append(cache2)
            
        return ys, caches

    def train_step_bwd(self, X_train, y_train, ys, caches):

        # Error
        loss, dys, grads_ = 0.0, [], []
        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.models[0], y_pred, y, lam=0)/ y_train.shape[0] # last layer
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Grads
        dh_next = np.zeros((1, self.H))
        grads = {k: np.zeros_like(v) for k, v in self.models[0].items()}
#         dh_next2 = np.zeros((1, self.H))
#         grads2 = {k: np.zeros_like(v) for k, v in self.models[1].items()} # for the 2nd layer
        for t in reversed(range(len(dys))):
#             grad2, dh_next2, dX = self.backward(dys[t], dh_next2, caches[t+1]) # self.model[1] == 2nd layer
            grad, dh_next, dX = self.backward(dys[t], dh_next, caches[t]) # self.model[0] == 1st layer
            for k in grads.keys():
                grads[k] += grad[k]
#                 grads2[k] += grad2[k]                
        grads_.append(grads)
#         grads_.append(grads2)

        return grads_, loss

In [144]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [145]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    M_, R_ = [], []
    for l in range(len(nn.models)): # l stands for layer number
        M = {k: np.zeros_like(v) for k, v in nn.models[l].items()}
        R = {k: np.zeros_like(v) for k, v in nn.models[l].items()}
        M_.append(M)
        R_.append(R)
    beta1 = .9
    beta2 = .999

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))

        #             # Testing can be completed once we make sure the training is done and is validated.
        #             sample = nn.test_step_fwd(X_mini[0], state)
        #             print(sample)

        ys_, caches_ = nn.train_step_fwd(X_mini, y_mini, state)
        grads_, loss = nn.train_step_bwd(X_mini, y_mini, ys_, caches_)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for l in range(len(nn.models)):
            for k in nn.models[0].keys():
                M_[l][k] = util.exp_running_avg(M_[l][k], grads_[l][k], beta1)
                R_[l][k] = util.exp_running_avg(R_[l][k], grads_[l][k]**2, beta2)

                m_k_hat = M_[l][k] / (1. - beta1**(t))
                r_k_hat = R_[l][k] / (1. - beta2**(t))

                nn.models[l][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [146]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.3698
Iter-2000 loss: 2.6147
Iter-3000 loss: 2.1153
Iter-4000 loss: 1.7936
Iter-5000 loss: 1.5697
Iter-6000 loss: 1.3930
Iter-7000 loss: 1.2623
Iter-8000 loss: 1.1591
Iter-9000 loss: 1.0733
Iter-10000 loss: 0.9972
Iter-11000 loss: 0.9389
Iter-12000 loss: 0.8755
Iter-13000 loss: 0.8362
